# ML Pipeline Preparation
Here I create a machine learning pipeline.
### 1. Import libraries and load data from database.
The steps I will folow will be: 
- Import the necessary Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [144]:
# import libraries
import numpy as np
import pandas as pd
import re
import sqlalchemy 

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import *
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package words to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [145]:
print(f'The nltk version is {nltk.__version__}.')
# print('The scikit-learn version is {}.'.format(sklearn.__version__))
print(f'The sklearn version is {sklearn.__version__}')
# print(f'The version of pickle is {pickle.__version__}')

The nltk version is 3.4.4.
The sklearn version is 0.21.2


In [146]:
#I load the data from the database
#sqlite:///
engine = sqlalchemy.create_engine('sqlite:///data/DisasterResponse.db')

#read it into a Pandas DataFrame
df = pd.read_sql_table('disaster', engine) 

In [147]:
#inspect
df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26172,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26173,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26174,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26175,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
df.shape

(26177, 40)

So there are 26,177 observations and 40 variables, just like in the last notebook. Which is good. 

Starting with the fifth (zero based indexing) column, 'request', till the 40th, the variables represent a one-hot encoding of message classifications. There are qualitative variables, 'genre' and 'related', that are we are going to leave out of our analysis for the time being. The 'original' messages is just that, the original message, often in a foreign language. 

Now we get the value counts of the qualitative variables:

In [152]:
for i in ['genre','related']:
    print(f'the value counts for "{i}" are:\n{df[i].value_counts()}\n')

the value counts for "genre" are:
news      13036
direct    10747
social     2394
Name: genre, dtype: int64

the value counts for "related" are:
1    19873
0     6117
2      187
Name: related, dtype: int64



#### Split the data into independent and dependent variables
For the independent variable we are going to use the message itself. For the dependent variable we are going to use the 35 categorizations of the messages. Thus the usual situation where there are numerous independent variable and one dependent variable is, for the time being, reversed. We will subsequently, in the process of turning the text content of 'message' into analyzable data, greatly increase the dimensionality of the independent variable.

In [153]:
X = df.loc[:,'message']
y = df.iloc[:,5:]

In [154]:
X.shape

(26177,)

In [155]:
y.shape

(26177, 35)

In [156]:
type(X)

pandas.core.series.Series

In [157]:
y.columns.values

array(['request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [158]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
pstemmer = nltk.PorterStemmer()

In [159]:
import re
url_regex = 'http[s]?://(?:[a-zA-Z][0-9]|[$-_@.&+]|[!*\(\),]|[?:%[0-9a-fA-F][0-9a-fA-F])'


In [160]:
def tokenize(text):
    #remove urs and replace with placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'url_placeholder')
    
    #normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #stem the works
    tokens = [pstemmer.stem(word) for word in tokens]
    
    #lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [161]:
X_tokenized = X.apply(lambda x: tokenize(x))

In [162]:
print(X_tokenized.head())
print(type(X_tokenized))
print(X_tokenized.index)
print(X_tokenized.values[:2])
print(X_tokenized.shape)

0    [weather, updat, cold, front, cuba, could, pas...
1                                           [hurrican]
2                                 [look, someon, name]
3    [un, report, leogan, 80, 90, destroy, onli, ho...
4    [say, west, side, haiti, rest, countri, today,...
Name: message, dtype: object
<class 'pandas.core.series.Series'>
RangeIndex(start=0, stop=26177, step=1)
[list(['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'])
 list(['hurrican'])]
(26177,)


We won't use `X_tokenized` in the analysis but create it to better understand what is going on. So we see that `X_tokenized` is a list pandas Series of lists, each list containing the words in a tweet. Since the list is still just counted as one object the size of the data set has not increased. We can see the effects of lemmatization and stemming in words like 'hurrican' and 'updat'. This will reduce the number of words that the algorithm has to deal will. Lowering the dimensionality of the data set will reduce the noise that that goes into the model. 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [163]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list[0]:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            
            
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [164]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

I have not been able to get the `FeatureUnion` feature to work so I am leaving it out for now, though I would like to include it in future versions. The `StartingVerbExtractor` doesn't seem all that vital but I could be wrong. Also, I would like to include the quantitative variables 'genre' and 'related' in the analysis, though I don't think that will require `FeatureUnion`. 

In [165]:
# pipeline1 = Pipeline([
#     ('features', FeatureUnion([

#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
#     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [167]:
pipeline.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [168]:
y_pred = pipeline.predict(X_test)

In [169]:
accuracy_score(y_test, y_pred)

0.3838044308632544

So this dry run of running random forest without any parameter tuning does just 38% accuracy. That is not very good, though there were 35 categories into which a message could be categorized. 

### 6. Improve your model
Use grid search to find better parameters. 

In [171]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [66]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [67]:
parameters = {
        'tfidf__norm': ['l2'],
        'tfidf__sublinear_tf': [False],
        'tfidf__smooth_idf': [False],
        'clf__estimator__n_estimators': [20],
        'clf__estimator__criterion' :['gini'],
        'clf__estimator__max_depth' : [1],
        'clf__estimator__max_features': ['log2']
    }

cv = GridSearchCV(pipeline, parameters)

Now fit the model

In [68]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [69]:
y_pred = cv.predict(X_test)
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 1,
 'clf__estimator__max_features': 'log2',
 'clf__estimator__n_estimators': 20,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False,
 'tfidf__sublinear_tf': False}

In [102]:
accuracy_score(y_test, y_pred)

0.4423223834988541

In [195]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_test.values[:,i], y_pred[:,i]))
    if i == 4:
        break

request
[[5415    0]
 [1130    0]]
offer
[[6513    0]
 [  32    0]]
aid_related
[[3810    1]
 [2727    7]]
medical_help
[[6010    0]
 [ 535    0]]
medical_products
[[6227    0]
 [ 318    0]]


In [108]:
#import xgboost as xgb

In [109]:
#clf = xgb.XGBClassifier()

In [124]:
# pipeline = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(xgb.XGBClassifier()))
# ])

In [138]:
# parameters = {
#         'vect__max_features': [1000, 1500, 2000],
#         'tfidf__norm': ['l2'],
#         'tfidf__sublinear_tf': [ False],
#         'tfidf__smooth_idf': [False],
#         'clf__estimator__max_depth': [3,5]
#     }

# cv = GridSearchCV(pipeline, parameters)

In [139]:
# cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [140]:
# pred = cv.predict(X_test)

In [141]:
# cv.best_params_

{'clf__estimator__max_depth': 3,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False,
 'tfidf__sublinear_tf': False,
 'vect__max_features': 1000}

In [142]:
# accuracy_score(y_test, y_pred)

0.3915966386554622

In [143]:
# for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_test.values[:,i], y_pred[:,i] ))

request
[[5288  127]
 [ 654  476]]
offer
[[6513    0]
 [  32    0]]
aid_related
[[3288  523]
 [1140 1594]]
medical_help
[[5984   26]
 [ 497   38]]
medical_products
[[6222    5]
 [ 293   25]]
search_and_rescue
[[6363    2]
 [ 175    5]]
security
[[6425    3]
 [ 117    0]]
military
[[6332    8]
 [ 190   15]]
child_alone
[[6545]]
water
[[6100   15]
 [ 302  128]]
food
[[5766   64]
 [ 394  321]]
shelter
[[5891   56]
 [ 443  155]]
clothing
[[6453    5]
 [  76   11]]
money
[[6400    4]
 [ 134    7]]
missing_people
[[6480    0]
 [  65    0]]
refugees
[[6298    1]
 [ 231   15]]
death
[[6222    9]
 [ 259   55]]
other_aid
[[5646   41]
 [ 821   37]]
infrastructure_related
[[6074    6]
 [ 462    3]]
transport
[[6250   10]
 [ 271   14]]
buildings
[[6181    9]
 [ 327   28]]
electricity
[[6419    1]
 [ 118    7]]
tools
[[6510    0]
 [  35    0]]
hospitals
[[6468    0]
 [  77    0]]
shops
[[6499    0]
 [  46    0]]
aid_centers
[[6460    1]
 [  84    0]]
other_infrastructure
[[6233    6]
 [ 305    1]]
w

### 9. Export your model as a pickle file

In [49]:
disaster_model = 'models/finalized_model.sav'
pickle.dump(cv, open(disaster_model, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.